In [2]:
!pip install protobuf


Defaulting to user installation because normal site-packages is not writeable


In [8]:
!pip install gtfs-realtime-python



Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement gtfs-realtime-python (from versions: none)
ERROR: No matching distribution found for gtfs-realtime-python


## Ce script récupère en temps réel les TripUpdate

| Champ                           | Supporté | Commentaires                                      |
|----------------------------------|----------|---------------------------------------------------|
| trip (TripDescriptor)           |          |                                                   |
| trip_id                         | Oui      |                                                   |
| route_id                        | Oui      |                                                   |
| direction_id                    | Oui      | S’il est disponible dans le flux statique.        |
| start_time                      | Oui      |                                                   |
| start_date                      | Oui      |                                                   |
| schedule_relationship (ScheduleRelationship) | Oui  | Les valeurs prises en charge sont les suivantes : PRÉVUE, ANNULÉ |
| véhicule (Descripteur de véhicule) | Oui    |                                                   |
| id                               | Oui      |                                                   |
| étiquette                        | Oui      |                                                   |
| trip_headsign si disponible dans le flux statique. | Oui |                                                   |
| license_plate                   | Oui      | Disponibilité non garantie.                       |
| stop_time_update (StopTimeUpdate) | Oui     |                                                   |
| stop_sequence                   | Non      |                                                   |
| stop_id                         | Oui      |                                                   |
| arrivée (StopTimeEvent)         | Oui      |                                                   |
| retarder                        | Non      |                                                   |
| Heure                           | Oui      |                                                   |
| incertitude                     | Non      |                                                   |
| départ (StopTimeEvent)          | Oui      |                                                   |
| retarder                        | Non      |                                                   |
| Heure                           | Oui      |                                                   |
| incertitude                     | Non      |                                                   |
| schedule_relationship (ScheduleRelationship) | Non  | Les valeurs prises en charge sont les suivantes : PRÉVUE, NO_DATA si l’arrêt est fermé, Écrémé en cas d’annulation partielle du voyage, NO_DATA et les arrêts SKIPPED ne peuvent apparaître que dans les trajets en cours |
| horodatage                      | Oui      |                                                   |
| retarder                        | Non      |                                                   |


In [2]:
import os
import base64
import requests
import json
from google.transit import gtfs_realtime_pb2  
from google.protobuf.json_format import MessageToJson
from kafka import KafkaProducer

os.environ["CLIENT_ID"] = "6722072041100899" 
os.environ["CLIENT_SECRET"] = "ISo0oqPMkfCB8wvaPSxrfj20EmnuvKwE" 

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

if not client_id or not client_secret:
    print("Error: CLIENT_ID and CLIENT_SECRET must be set as environment variables.")
    exit()

auth_string = f"{client_id}:{client_secret}"
encoded_auth = base64.b64encode(auth_string.encode()).decode()

city = "tampere"  
endpoint = "tripupdate"  
url = f"https://data.waltti.fi/{city}/api/gtfsrealtime/v1.0/feed/{endpoint}"

headers = {
    "Authorization": f"Basic {encoded_auth}",
}

kafka_config = {
    'bootstrap_servers': 'localhost:9092',
}
producer = KafkaProducer(
    **kafka_config,
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

kafka_topic = "tripupdate"  

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    feed = gtfs_realtime_pb2.FeedMessage()
    feed.ParseFromString(response.content)
    feed_json = json.loads(MessageToJson(feed))

    if feed.entity:
        first_entity = feed.entity[0]
        vehicle_json = json.loads(MessageToJson(first_entity))
        print(json.dumps(vehicle_json, indent=4))
    else:
        print("Aucune entité trouvée.")

    for entity in feed.entity:
        vehicle_json = json.loads(MessageToJson(entity))
        producer.send(kafka_topic, vehicle_json)

    producer.flush()

except requests.exceptions.RequestException as e:
    print(json.dumps({"error": f"Error connecting to the API: {e}"}, indent=4))
except Exception as e:
    print(json.dumps({"error": f"Unexpected error: {e}"}, indent=4))
finally:
    producer.close()

{
    "id": "74_15266_9085705_2024-12-20_3_0_23:58:00",
    "tripUpdate": {
        "trip": {
            "tripId": "74_15266_9085705",
            "startTime": "23:58:00",
            "startDate": "20241220",
            "scheduleRelationship": "SCHEDULED",
            "routeId": "356920",
            "directionId": 0
        },
        "stopTimeUpdate": [
            {
                "stopSequence": 13,
                "departure": {
                    "time": "1734733095"
                },
                "stopId": "0813",
                "scheduleRelationship": "SCHEDULED"
            },
            {
                "stopSequence": 14,
                "arrival": {
                    "time": "1734733137"
                },
                "departure": {
                    "time": "1734733171"
                },
                "stopId": "0811",
                "scheduleRelationship": "SCHEDULED"
            },
            {
                "stopSequence": 15,
                "

### Ce script récupère en temps réel les positions des véhicules de transport public à Tampere via l'endpoint vehicleposition de l'API GTFS Realtime de Waltti. 

| Champ                          | Supporté | Commentaires                                                                 |
|---------------------------------|----------|-----------------------------------------------------------------------------|
| trip (TripDescriptor)          |          |                                                                             |
| trip_id                         | Oui      |                                                                             |
| route_id                        | Oui      |                                                                             |
| direction_id                    | Oui      | S’il est disponible dans le flux statique.                                  |
| start_time                      | Oui      |                                                                             |
| start_date                      | Oui      |                                                                             |
| schedule_relationship (ScheduleRelationship) | Oui | Les valeurs prises en charge sont les suivantes : PRÉVUE                     |
| véhicule (Descripteur de véhicule) | Oui    |                                                                             |
| id                               | Oui      |                                                                             |
| étiquette                        | Oui      |                                                                             |
| trip_headsign si disponible dans le flux statique. | Oui |                                                                             |
| license_plate                   | Oui      | Disponibilité non garantie.                                                 |
| position (Position)            | Oui      |                                                                             |
| latitude                        | Oui      |                                                                             |
| longitude                       | Oui      |                                                                             |
| Attitude                        | Oui      |                                                                             |
| odomètre                        | Non      |                                                                             |
| vitesse                         | Oui      |                                                                             |
| current_stop_sequence           | Oui      |                                                                             |
| stop_id                         | Oui      |                                                                             |
| current_status                  | Oui      |                                                                             |
| horodatage                      | Oui      |                                                                             |
| congestion_level (CongestionLevel) | Oui    | La valeur prise en charge est la suivante : UNKNOWN_CONGESTION_LEVEL        |
| occupancy_status (OccupancyStatus) | Non    |                                                                             |


In [2]:
import os
import base64
import requests
import json
from google.transit import gtfs_realtime_pb2  
from google.protobuf.json_format import MessageToJson
from kafka import KafkaProducer

os.environ["CLIENT_ID"] = "6722072041100899" 
os.environ["CLIENT_SECRET"] = "ISo0oqPMkfCB8wvaPSxrfj20EmnuvKwE" 

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

if not client_id or not client_secret:
    print("Error: CLIENT_ID and CLIENT_SECRET must be set as environment variables.")
    exit()

auth_string = f"{client_id}:{client_secret}"
encoded_auth = base64.b64encode(auth_string.encode()).decode()

city = "tampere"  
endpoint = "vehicleposition"  
url = f"https://data.waltti.fi/{city}/api/gtfsrealtime/v1.0/feed/{endpoint}"

headers = {
    "Authorization": f"Basic {encoded_auth}",
}

kafka_config = {
    'bootstrap_servers': 'localhost:9092',
}
producer = KafkaProducer(
    **kafka_config,
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

kafka_topic = "vehicleposition"  

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    feed = gtfs_realtime_pb2.FeedMessage()
    feed.ParseFromString(response.content)
    feed_json = json.loads(MessageToJson(feed))

    if feed.entity:
        first_entity = feed.entity[0]
        vehicle_json = json.loads(MessageToJson(first_entity))
        print(json.dumps(vehicle_json, indent=4))
    else:
        print("Aucune entité trouvée.")

    for entity in feed.entity:
        vehicle_json = json.loads(MessageToJson(entity))
        producer.send(kafka_topic, vehicle_json)

    producer.flush()

except requests.exceptions.RequestException as e:
    print(json.dumps({"error": f"Error connecting to the API: {e}"}, indent=4))
except Exception as e:
    print(json.dumps({"error": f"Unexpected error: {e}"}, indent=4))
finally:
    producer.close()

{
    "id": "47374_43",
    "vehicle": {
        "trip": {
            "tripId": "74_15625_9183974",
            "startTime": "13:15:00",
            "startDate": "20241226",
            "scheduleRelationship": "SCHEDULED",
            "routeId": "2947374",
            "directionId": 1
        },
        "position": {
            "latitude": 61.49115,
            "longitude": 23.856312,
            "bearing": 127.0,
            "speed": 3.5833333
        },
        "currentStopSequence": 40,
        "currentStatus": "IN_TRANSIT_TO",
        "timestamp": "1735214644",
        "congestionLevel": "UNKNOWN_CONGESTION_LEVEL",
        "stopId": "5070",
        "vehicle": {
            "id": "47374_43",
            "label": "Lamminrahka",
            "licensePlate": "CMR-472"
        }
    }
}


#### **Ce script récupère en temps réel les servicealert**

| Champ                                | Supporté | Commentaires                                                          |
|--------------------------------------|----------|----------------------------------------------------------------------|
| active_period (Plage horaire)        | Oui      |                                                                      |
| commencer                            | Oui      |                                                                      |
| fin                                  | Oui      |                                                                      |
| informed_entity (EntitySelector)    | Oui      |                                                                      |
| agency_id                            | Oui      |                                                                      |
| route_id                             | Oui      |                                                                      |
| route_type                           | Oui      |                                                                      |
| voyage                               | Non      |                                                                      |
| stop_id                              | Oui      |                                                                      |
| cause (Cause)                        | Oui      |                                                                      |
| effect (Effet)                       | Oui      |                                                                      |
| url (TranslatedString)               | Oui      |                                                                      |
| Paramétrable                         |          |                                                                      |
| header_text (TranslatedString)       | Oui      |                                                                      |
| description_text (TranslatedString)  | Oui      |                                                                      |
| Niveau de gravité                    | Oui      | Valeurs prises en charge : INFO, AVERTISSEMENT, SÉVÈRE               |


In [8]:
import os
import base64
import requests
import json
from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import MessageToJson
from kafka import KafkaProducer

os.environ["CLIENT_ID"] = "6722072041100899"
os.environ["CLIENT_SECRET"] = "ISo0oqPMkfCB8wvaPSxrfj20EmnuvKwE"

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

if not client_id or not client_secret:
    print("Error: CLIENT_ID and CLIENT_SECRET must be set as environment variables.")
    exit()

auth_string = f"{client_id}:{client_secret}"
encoded_auth = base64.b64encode(auth_string.encode()).decode()

city = "tampere"
endpoint = "servicealert"
url = f"https://data.waltti.fi/{city}/api/gtfsrealtime/v1.0/feed/{endpoint}"

headers = {
    "Authorization": f"Basic {encoded_auth}",
}

kafka_topic = "servicealert"
producer = KafkaProducer(
    bootstrap_servers='localhost:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

def delivery_report(future):
    try:
        record_metadata = future.get(timeout=10)
        print(f"Message delivered to {record_metadata.topic} [partition: {record_metadata.partition}]")
    except Exception as e:
        print(f"Message delivery failed: {str(e)}")

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    feed = gtfs_realtime_pb2.FeedMessage()
    feed.ParseFromString(response.content)
    feed_json = json.loads(MessageToJson(feed))

    print(json.dumps(feed_json["entity"][0], indent=4))

    for entity in feed.entity:
        entity_json = json.loads(MessageToJson(entity))
        future = producer.send(
            kafka_topic,
            value=entity_json
        )
        delivery_report(future)

    producer.flush()

except requests.exceptions.RequestException as e:
    print(json.dumps({"error": f"Error connecting to the API: {e}"}, indent=4))
except Exception as e:
    print(json.dumps({"error": f"Unexpected error: {e}"}, indent=4))


{
    "id": "14196",
    "isDeleted": false,
    "alert": {
        "activePeriod": [
            {
                "start": "1721703600",
                "end": "1739584740"
            }
        ],
        "informedEntity": [
            {
                "stopId": "3569"
            }
        ],
        "cause": "CONSTRUCTION",
        "effect": "NO_SERVICE",
        "url": {
            "translation": [
                {
                    "text": "",
                    "language": "fi"
                },
                {
                    "text": "",
                    "language": "en"
                }
            ]
        },
        "headerText": {
            "translation": [
                {
                    "text": "Pys\u00e4kki suljettu: Hervanta",
                    "language": "fi"
                },
                {
                    "text": "Bus stop closed: Hervanta",
                    "language": "en"
                }
            ]
        },
        

In [2]:
!pip install requests

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import requests

def get_weather(api_key, lat, lon, units="metric", lang="en"):
    """
    Obtenez les données météorologiques actuelles à partir de l'API OpenWeather.

    :param api_key: Votre clé API OpenWeather.
    :param lat: Latitude de l'emplacement.
    :param lon: Longitude de l'emplacement.
    :param units: Unités de mesure (standard, metric ou imperial).
    :param lang: Langue de la réponse (par exemple : en, fr).
    :return: Les données météo sous forme de dictionnaire.
    """
    base_url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "lat": lat,
        "lon": lon,
        "appid": api_key,
        "units": units,
        "lang": lang
    }
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Vérifie les erreurs HTTP
        return response.json()  # Retourne les données sous forme de dictionnaire
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de l'appel API : {e}")
        return None

# Exemple d'utilisation
if __name__ == "__main__":
    # Remplacez par votre clé API
    API_KEY = "c79edb5700b1fdd5ee28688166555292"
    LAT = 44.34  # Exemple de latitude
    LON = 10.99  # Exemple de longitude
    
    weather_data = get_weather(API_KEY, LAT, LON, units="metric", lang="fr")
    
    if weather_data:
        print("Données météo :")
        print(f"Ville : {weather_data.get('name')}")
        print(f"Description : {weather_data['weather'][0]['description']}")
        print(f"Température : {weather_data['main']['temp']}°C")
        print(f"Humidité : {weather_data['main']['humidity']}%")
        print(f"Vitesse du vent : {weather_data['wind']['speed']} m/s")
    else:
        print("Impossible de récupérer les données météo.")


Erreur lors de l'appel API : 401 Client Error: Unauthorized for url: https://api.openweathermap.org/data/2.5/weather?lat=44.34&lon=10.99&appid=c79edb5700b1fdd5ee28688166555292&units=metric&lang=fr
Impossible de récupérer les données météo.


### Meteo

In [7]:
import requests

# Paramètres
lat = 61.482771
lon = 23.773996 
exclude = "minutely,hourly"
api_key = "c79edb5700b1fdd5ee28688166555292"
url = f"https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&exclude={exclude}&units=metric&lang=fr&appid={api_key}"

# Appel à l'API
response = requests.get(url)

if response.status_code == 200:
    print("Données météo :", response.json())
else:
    print(f"Erreur {response.status_code} : {response.text}")


Erreur 401 : {"cod":401, "message": "Please note that using One Call 3.0 requires a separate subscription to the One Call by Call plan. Learn more here https://openweathermap.org/price. If you have a valid subscription to the One Call by Call plan, but still receive this error, then please see https://openweathermap.org/faq#error401 for more info."}
